# EJERCICIO CNNS

Teneis que hacer un modelo de ___convulsiones___. El dataset lo teneis aqui:

https://www.kaggle.com/c/cifar-10/data

IMPORTANTE: No useis el test.7z. Descargaos solo el train.7z y las train_labels. Despues ya dividis en train-test y con ese test ya sacais las metricas finales.

QUE OS PIDO:
- Descargad el dataset
- Leedlo en el NB (podeis investigar como hacerlo o usad el NB de landscapes como punto de partida)
- Haced train-test y escalar si procede
- Montad un modelo que tenga: Al menos 3 bloques convolucionales (Conv2D+Pooling+Dropout). PERO TIENE QUE SER UN MODELO FUNCIONAL (https://keras.io/guides/functional_api/). Es una chorrada, pero teneis que investigar la sintaxis nueva.
- Clasificacion, 10 clases, 0.1 de validation split... ya sabeis.
- Entrenad con los parametros que querais e id jugando con el numero de epocas, dropout, o incluso mas o menos capas, numero de filtros... etc... (lo hemos visto todo en clase)
- Evaluad el modelo con las metricas que creais convenientes


NOTAS: 
- Las imagenes son de 32x32
- Son a color luego el shape ya sabeis...
- Recordad que la primera capa debe llevar el input_shape
- Si haceis una red muy grande puede ir lento, asi que empezad despacito y creced si procede
- Usad la historia de epocas (train_loss vs val_loss) para ver si estais overfiteando
- No es imprescindible pero muy muy muy recomendable que hagais el ejercicio en un google colab. Tendreis que ver como subir los datos etc.. pero las epocas iran muuucho mas rapidas y asi aprendeis. Podeis hacerlo local, ver que funciona y migrar a colab, o directamente hacerlo todo en colab
- En teoria, salvo que hay que declarar el modelo Functional, todo lo hemos visto ya. Intentad probar cosas e intentad no abusar de Chatgpt. Daos la oportunidad de hacerlo vosotros y demostraros que podeis. 

- Ah! El ejercicio es OBLIGATORIO. Teneis 3 dias y una tarde para hacerlo, no hay excusas. Todos los aprendizajes de hacer este NB los voy a dar por sabidos. No lo corregiré en clase, pero resolveré todas las dudas que traigais, eso si.

Si durante el finde os surgen dudas me escribis. Pero me gustaria que intentarais resolverlo con la maxima autonomia. Pero escribidme si os atascais.

### Description

CIFAR-10  is an established computer-vision dataset used for object recognition. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 color images containing one of 10 object classes, with 6000 images per class. It was collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.


The CIFAR-10 data consists of 60,000 32x32 color images in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images in the official data. We have preserved the train/test split from the original dataset. 

 The provided files are:

- train.7z - a folder containing the training images in png format
- trainLabels.csv - the training labels

The label classes in the dataset are:

- airplane 
- automobile 
- bird 
- cat 
- deer 
- dog 
- frog 
- horse 
- ship 
- truck

The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.

In [43]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2 
import tensorflow as tf

In [26]:
LABELS_CSV = "trainLabels.csv"
IMAGE_SIZE = (32,32)
TRAIN_PATH = "train"

In [23]:
names = pd.read_csv("trainLabels.csv")
class_names = (names["label"].unique()).tolist()
class_names_labels = {class_names:i for i, class_names in enumerate(class_names)}
class_names_labels

{'frog': 0,
 'truck': 1,
 'deer': 2,
 'automobile': 3,
 'bird': 4,
 'horse': 5,
 'ship': 6,
 'cat': 7,
 'dog': 8,
 'airplane': 9}

In [39]:
def read_data(path, im_size, labels_csv):
    X = []
    y = []
    
    # Leer etiquetas del archivo CSV
    labels_df = pd.read_csv(labels_csv)
    
    # Crear el diccionario de etiquetas
    labels_dict = {f"{row['id']}.png": row['label'] for _, row in labels_df.iterrows()}
    
    # Filtrar solo los archivos presentes en el directorio
    valid_files = set(os.listdir(path))
    labels_dict = {k: v for k, v in labels_dict.items() if k in valid_files}
    
    print(f"Total de archivos a procesar: {len(valid_files)}")

    # Procesar imágenes con un contador
    for idx, file in enumerate(valid_files):
        image_path = os.path.join(path, file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, im_size)
        X.append(image)
        y.append(labels_dict[file])  # Asignar etiqueta según el archivo CSV
        
        # Mostrar progreso cada 1000 imágenes
        if (idx + 1) % 1000 == 0:
            print(f"Procesadas {idx + 1} imágenes...")

    print("Carga completada.")
    return np.array(X), np.array(y)


In [45]:
X, y = read_data(TRAIN_PATH, IMAGE_SIZE, LABELS_CSV)

Total de archivos a procesar: 50000
Procesadas 1000 imágenes...
Procesadas 2000 imágenes...
Procesadas 3000 imágenes...
Procesadas 4000 imágenes...
Procesadas 5000 imágenes...
Procesadas 6000 imágenes...
Procesadas 7000 imágenes...
Procesadas 8000 imágenes...
Procesadas 9000 imágenes...
Procesadas 10000 imágenes...
Procesadas 11000 imágenes...
Procesadas 12000 imágenes...
Procesadas 13000 imágenes...
Procesadas 14000 imágenes...
Procesadas 15000 imágenes...
Procesadas 16000 imágenes...
Procesadas 17000 imágenes...
Procesadas 18000 imágenes...
Procesadas 19000 imágenes...
Procesadas 20000 imágenes...
Procesadas 21000 imágenes...
Procesadas 22000 imágenes...
Procesadas 23000 imágenes...
Procesadas 24000 imágenes...
Procesadas 25000 imágenes...
Procesadas 26000 imágenes...
Procesadas 27000 imágenes...
Procesadas 28000 imágenes...
Procesadas 29000 imágenes...
Procesadas 30000 imágenes...
Procesadas 31000 imágenes...
Procesadas 32000 imágenes...
Procesadas 33000 imágenes...
Procesadas 34000

In [ ]:
# Convertimos las etiquetas a índices
y = np.array([class_names_labels[label] for label in y])

In [55]:
# Normalizamos las imagenes
X = X/255

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (40000, 32, 32, 3)
y_train shape: (40000,)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000,)


In [57]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

In [58]:
# Definimos la capa de entrada
# CIFAR-10 tiene imágenes de 32x32 con 3 canales de color (RGB)
input_layer = Input(shape=(32,32,3))

## **Bloques convolucionales**

1. La capa convolucinal (Conv2D) es la capa fundamental de una CNN. Esta capa aplica filtros(kernels) sobre la imagen de entrada para extraer características como bordes, formas y texturas. La convolución hace que el modelo aplique estos filtros sobre la imagen, realizando una multiplicación con los valores de la imagen, sumándolos para crear un mapa de activación. Cada filtro se entrena para aprender características específicas de los datos y esti permite a la red capturar patrones espaciales.
```python
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
```

- **32** es el número de filtros que se aplican a la imagen y cada filtro tiene un tamaño de **(3,3)** y genera un mapa de activación que captura diferentes patrones.
- **activation = 'relu'** es la función que se aplica después de la convolución, esto introduce no linealidad en el modelo ayudando a aprender patrones más complejos.
- **padding = 'same'** asegura que las dimensiones de salida sean las mismas que las de entrada. Si no lo usamos la imagen reduciría su tamaño.

2. La capa de pooling (MaxPooling2D) se utiliza para reducir las dimensiones espaciales(alto y ancho) de las imágenes, sin perder información importante. Esto se logra mediante la selección de los valores más altos de un conjunto de píxeles vecinos. Es importante porque reduce la complejidad computacional y ayuda a que el modelo sea menos sensible a pequeñas traslaciones y desplazamientos en las imagenes(invarianza a pequeñas transformaciones).
```python
x = MaxPooling2D((2, 2))(x)
```

- **(2, 2)** es el tamaño de la ventana de pooling, significa que en este caso tomamos el valor máximo de cada bloque de 2x2 píxeles. La salida de esta operación trendrá la mitad de tamaño en cada dimensión (alto y ancho) en comparación con la entrada.

3. La capa dropout (Dropout) es una técnica de regularización que desactiva aleatoriamente un porcentaje de las neuronas durante el entrenamiento. Esto ayuda a prevenir el overfitting, ya que evita que el modelo dependa demasiado de neuronas espacíficas. Es importante para que el modelo generalice bien.
```python
x = Dropout(0.25)(x)
```

- **0.25** esto significa que el 25% de las neuronas de desactivarán aleatoriamente durante cada paso del entrenamiento.

# Primer bloque convolucional
```python
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)  # 33, 33, 32 (dimensión)
```


Conv2D(32, (3, 3), activation='relu', padding='same'): Aquí se define una capa convolucional 2D. Los parámetros de esta capa son:

32: El número de filtros o kernels que se aplicarán a la entrada. Cada filtro aprende características diferentes de la imagen.
(3, 3): El tamaño del filtro es de 3x3 píxeles.
activation='relu': Se aplica la función de activación ReLU (Rectified Linear Unit), que transforma los valores negativos a cero y deja los positivos sin cambio. Esto introduce no linealidad en el modelo.

padding='same': Significa que se aplicará un "relleno" (padding) para mantener las dimensiones de la imagen de entrada constantes después de la convolución. Es decir, si la imagen de entrada tiene dimensiones H×W×C (alto, ancho, canales), la salida tendrá las mismas dimensiones en altura y anchura, solo que con un número de canales diferente (32 en este caso).

Resultado: Si la entrada tenía dimensiones (33, 33, 3) (por ejemplo, una imagen RGB de 33x33 píxeles), la salida tendrá dimensiones (33, 33, 32). La salida de esta capa tendrá 32 canales (uno por cada filtro).

```python
x = MaxPooling2D((2,2))(x) #16, 16, 32
```

MaxPooling2D((2, 2)): Esta capa realiza una operación de "max pooling", que reduce las dimensiones espaciales de la imagen (alto y ancho). En este caso, se toma una ventana de tamaño 2x2 y, de cada ventana, se selecciona el valor máximo.

Esto reduce la resolución espacial de la imagen, ayudando a extraer características más robustas y reduciendo la cantidad de parámetros.
Al aplicar el max pooling sobre una entrada de tamaño (33, 33, 32), la salida tendrá dimensiones (16, 16, 32), ya que el max pooling reduce a la mitad las dimensiones espaciales (33/2 ≈ 16).

Resultado: La salida tiene dimensiones (16, 16, 32), es decir, se ha reducido el tamaño en la dirección de alto y ancho, pero el número de canales sigue siendo 32.

```python
x = Dropout(0.25)(x)
```

Dropout(0.25): El "dropout" es una técnica de regularización que ayuda a prevenir el sobreajuste (overfitting) durante el entrenamiento. Durante el entrenamiento, se "apagan" aleatoriamente el 25% de las unidades (neuronas) en la capa. Esto significa que el 25% de los valores en la salida de esta capa se establecerán a cero en cada paso de entrenamiento, lo que fuerza a la red a no depender demasiado de ningún conjunto específico de neuronas.

Este valor de 0.25 significa que el 25% de las conexiones de esta capa serán eliminadas aleatoriamente.
Resultado: La salida sigue teniendo las mismas dimensiones que antes de aplicar dropout (16, 16, 32), pero algunas de las activaciones se habrán puesto a cero de manera aleatoria.



In [59]:
# Primer bloque convolucional
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_layer) #33, 33, 32 (dimensión)
x = MaxPooling2D((2,2))(x) #16, 16, 32
x = Dropout(0.25)(x)

In [61]:
# Segundo bloque concolucional 
x = Conv2D(64, (3,3), activation='relu', padding='same')(x) #16, 16, 64
x = MaxPooling2D((2,2))(x) #8, 8 64
x = Dropout(0.25)(x)

In [62]:
# Tercer bloque convolucional
x = Conv2D(128, (3,3), activation='relu', padding='same')(x) #8, 8, 128
x = MaxPooling2D((2,2))(x) #4, 4, 128
x = Dropout(0.25)(x)

In [ ]:
# Las capas convolucionales generan datos 3D, antes de pasar a las capas densas aplanamos los datos
x = Flatten()(x)

## **Capas densas y capa de salida**

### **Capas densas**

Después de las capas convolucionales, el modelo ha aprendido patrones locales de la imagen como bordes, texturas o combinaciones más complejas. Sin embargo, estos patrones deben combinarse para tomar decisiones sobre la clase imagen. 

La capa densa conecta todas las neurones de la capa anterior con cada neurona de la capa densa. Esto se llama una "capa totalmente conectada" (fully connected layer).

En este caso, después de pasar por las capas convolucionales, tenemos una representación de la imagen en una forma reducida (por el pooling y a la activación de la convolucion), pero todavía en 3D. Necesitamos aplanar esta representación para convertirla en un vector 1D antes de poder pasarlo a una capa densa. Esto se hace mediante la operación **Flatten()**. Después de aplanar la capa densa ayuda a realizar las decisiones finales sobre la clasificación de la imagen. 

La capa densa tiene una gran capacidad de aprendizaje no lineal. Como cada neurona está conectada a todas las neuronas de la capa anterior, la capa densa puede combinar todas las características extraídas por las capas convolucionales para tomar decisiones complejas sobre la clase de la imagen.
```python
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
```

- **Flatten()** toma las salidas de las capas convolucionales (3D) y las aplana en un solo vector de valores. Por ejemplo, si la salida de las capas anteriores tiene una forma  (batch_size, 4, 4, 128), al aplicar Flatten() la forma se convierte en (batch_size, 2048).
- **Dense(128)** significa que la capa tiene 128 neuronas y cada una de ellas está conectada a todas las neuronas de la capa anterior (el vector aplanado). **activation='relu'** aplica una función de activación ReLU.

### **Capa de salida**

Después de pasar por la capa densa, necesitamos una capa final para clasificar la imagen en una de las 10 categorías de CIFAR-10. Esto lo realiza la capa de salida.

La capa de salida es una capa densa con tantas neuronas como clases de salida haya (en este caso 10), y utiliza la función de activación **softmax**. Esta función convierte las salidas de la capa densa en probabilidades, la clase con probabilidad más alta será la que se prediga como la clase de la imagen.

Esta capa es importante porque decide a qué clase pertenece la imagen.
```python
output_layer = Dense(10, activation='softmax')(x)
```

- **Dense(10)** significa que hay 10 neuronas, una por cada clase en CIFAR-10 (de 0 a 9). Esta capa decide en qué clase encaja mejor la imagen.
- **activation='softmax'** es la función que convierte las salidas de las neuronas en probabilidades. La clase con la probabilidad más alta será la predicción final del modelo.


In [64]:
# Añadimos capas densas
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)

output_layer = Dense(10, activation='softmax')(x) # CIFAR-10 tiene 10 clases

## **Creamos el modelo**

Cuando creamos el modelo en Keras, estamos construyendo toda la arquitectura de la red neuronal, es decir, estamos definiendo las capas que componen el modelo, cómo se conectan entre ellas y qué tipo de operaciones realizan. Este es el proceso de configurar el flujo de los datos que atraviesa las capas, desde la entrada (las imágenes) hasta la salida (las predicciones de las clases).

En este caso,  la creación del modelo se hace a través del modelo funcional. Lo que hacemos es definir las entradas, aplicar las capas que hemos configurado y, finalmente, obtener la salida que predice la clase de la imagen.

### **Qué significa el model.summary()**

El resumen que devuelve este comando proporciona una visión general de la arquitectura de la red neuronal que acabamos de construir. Este resumen es muy útil para entender cómo está configurada la red y verificar que todo está en orden. También es importante para comprobar el número de parámetros entrenables de cada capa.

1. **Capas Convolucionales (Conv2D)** en cada capa convolucional los parámetros que se entrenan son los pesos de los filtros y los sesgos.
```python
Parámetros en Conv2D=(f×f×canales de entrada+1)×número de filtros
```
- En el caso de la primera capa convolucional: tamaño del filtro = 3x3; canales de entrada = 3 (por ser una imagen RGB); número de filtros = 32; parámetros por filtro = (3x3x3+1) = 28; total de parámetros = 32.

2. **Capas de MaxPooling (MaxPooling2D)** no tienen parámetros entrenables, su función es reducir la dimensión espacial (alto y ancho) de las imágenes, pero no agregan pesos ni sesgos.

3. **Capas de Dropout (Dropout)** tampoco tienen parámetros, su objetivo es prevenir el overfitting.

4. **Capa Densa (Dense)** conectan todas las neuronas de la capa anterior con cada neurona de la capa siguiente. Los parámetros que se entrenan son los pesos de las conexiones y los sesgos.
```python 
Parámetros en Dense=(número de neuronas de la capa anterior+1)×número de neuronas de la capa densa
```

- Neuronas de la capa anterior = 2048 (salida de la capa Flatten); neuronas en la caoa densa = 128; parámetros = (2048 +1)x128 =262272.

5. **Capa de salida** es también una capa densa pero con 10 neuronas.

In [65]:
# Creamos el modelo
model = Model(inputs=input_layer, outputs=output_layer, name="CIFAR10_Model")

In [ ]:
# Revisamos la estructura del modelo
model.summary()

Model: "CIFAR10_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,810 (1.36 MB)

 Trainable params: 356,810 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Compilamos el modelo
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [72]:
# Entrenamos el modelo
history = model.fit(X_train, y_train, validation_split=0.1, epochs=2, batch_size=64, verbose=2)

Epoch 1/2
563/563 - 27s - 48ms/step - accuracy: 0.7921 - loss: 0.5843 - val_accuracy: 0.7800 - val_loss: 0.6751
Epoch 2/2
563/563 - 25s - 45ms/step - accuracy: 0.7988 - loss: 0.5695 - val_accuracy: 0.7755 - val_loss: 0.6789


In [73]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Precisión en test: {test_accuracy:.2f}")

313/313 - 2s - 7ms/step - accuracy: 0.7682 - loss: 0.6559
Precisión en test: 0.77
